In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

2023-05-23 12:35:22.182106: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 12:35:22.774746: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-23 12:35:22.822940: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jccarville/.pyenv/versions/3.10.6/envs/proposture_env/lib/python3.10/site-packages/cv2/../

In [2]:
cap = cv2.VideoCapture('../media/justin_pu.mp4')

# Pushup counter variables
counter = 0 
stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            # Break the loop if there are no more frames in the video
            break
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark

            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,
                          landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z]
            # print(l_shoulder)
            
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                          landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                          landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z]
            # print(l_hip)
            
            distance = l_shoulder - l_hip
            # print(distance)
            
            # Visualize right angle
            cv2.putText(image, str(distance), 
                           tuple(np.multiply(l_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
            )  
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[0.763126015663147, 0.3184421956539154, 0.011569950729608536]
[0.4671340584754944, 0.5483951568603516, 0.11973055452108383]
[0.7631741166114807, 0.3093205392360687, 0.008848491124808788]
[0.46447962522506714, 0.548088788986206, 0.11997001618146896]
[0.7631388306617737, 0.30665233731269836, 0.013796724379062653]
[0.4624979496002197, 0.5480822920799255, 0.12079700827598572]
[0.7633951902389526, 0.30585911870002747, 0.013804197311401367]
[0.46236759424209595, 0.5496623516082764, 0.12085185945034027]
[0.763801634311676, 0.3060585558414459, 0.01437115017324686]
[0.4619538187980652, 0.5512925386428833, 0.12101957947015762]
[0.7644323706626892, 0.306199848651886, 0.013774633407592773]
[0.4617151916027069, 0.555706799030304, 0.12124894559383392]
[0.7645998001098633, 0.3063867390155792, 0.01946404203772545]
[0.46002396941185, 0.5571165680885315, 0.1214594617486]
[0.764994204044342, 0.30651363730430603, 0.025420282036066055]
[0.4587840735912323, 0.5609440207481384, 0.12248530238866806]
[0.765755

In [19]:
#function to calculate angles
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return round(angle) 

In [42]:
import numpy as np
import cv2
import mediapipe as mp

cap = cv2.VideoCapture('../media/full_pushup.mp4')

# Pushup position variable
stage = None
quality = None
counter = 0

# Setup mediapipe instance
with mp.solutions.pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            # Break the loop if there are no more frames in the video
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
          
            ##get elbow angles
            # Get coordinates for left elbow
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # Calculate angle
            l_elbow_angle = calculate_angle(l_shoulder, l_elbow, l_wrist)
            
            # Visualize angle
            cv2.putText(image, str(l_elbow_angle), 
                           tuple(np.multiply(l_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA
                                )
            # Get coordinates for right elbow
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle
            r_elbow_angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
                                      
            # Visualize angle
            cv2.putText(image, str(r_elbow_angle), 
                           tuple(np.multiply(r_elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
            
            # Counter and State logic
            if l_elbow_angle and r_elbow_angle > 160:
                stage = "up"
            if l_elbow_angle and r_elbow_angle < 90 and stage =='up':
                stage="down"
                counter +=1 
                
            ##get shoulder angles
            # Get coordinates for left shoulder
            l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            
            # Calculate angle
            l_shoulder_angle = calculate_angle(l_hip, l_shoulder, l_elbow)
            
            # Visualize angle
            cv2.putText(image, str(l_shoulder_angle), 
                           tuple(np.multiply(l_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA
                                )
            # get coords for right shoulder
            r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            
            # Calculate angle
            r_shoulder_angle = calculate_angle(r_hip, r_shoulder, r_elbow)
            
            # Visualize angle
            cv2.putText(image, str(r_shoulder_angle), 
                           tuple(np.multiply(r_shoulder, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)    
            
            avg_shoulder_angle = round((l_shoulder_angle+r_shoulder_angle)/2)
                
            #  ##get the distance for elbow to hips
            # l_elbow = np.array([landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW].x,
            #                        landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW].y,
            #                        landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW].z])
            # print(l_elbow)
            # l_hip = np.array([landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP].x,
            #                   landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP].y,
            #                   landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP].z])
            # # print(l_hip)
            
            # # Calculate Euclidean distance
            # distance = np.linalg.norm(l_elbow - l_hip)
            # print(f"Distance: {distance}")  
            
            # # Visualize the distance
            # cv2.putText(image, f"Distance: {distance:.2f}", 
            #             tuple(np.multiply(l_shoulder[:2], [640, 480]).astype(int)), 
            #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2, cv2.LINE_AA)
            
            # Quality logic
            if abs(avg_shoulder_angle-45)>2:
                quality = 'BAD'
            elif abs(avg_shoulder_angle-45)<2:
                quality = 'GOOD'
            # if stage == "down" and abs(avg_shoulder_angle-45)<2:
            #     quality = "BAD"
            # elif stage == "down" and distance < .35:
            #     quality = 'GOOD' 
            
        except:
            pass
        
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Status data
        cv2.putText(image, 'Quality', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, quality, 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        # Render detections
        mp.solutions.drawing_utils.draw_landmarks(image, results.pose_landmarks, 
                                                  mp.solutions.pose.POSE_CONNECTIONS,
                                                  mp.solutions.drawing_utils.DrawingSpec(
                                                      color=(245, 117, 66), thickness=2, circle_radius=2),
                                                  mp.solutions.drawing_utils.DrawingSpec(
                                                      color=(245, 66, 230), thickness=2, circle_radius=2)
                                                  )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [34]:
l_elbow

array([ 0.67601359,  0.62536877, -0.39823201])

In [35]:
distance

0.4014838038897157